In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('labeled_dataset.csv')

df.head(5)

,LogName,Date,Time,ComputerName,SourceName,TaskCategory,EventType,Keywords,OpCode,EventCode,Type,RecordNumber,Message,isMalicious
0,Application,2023-09-22,00:10:15,CLIENT-PC,Microsoft-Windows-Security-SPP,NaN,4,Classic,NaN,16384,4,3267,Successfully scheduled Software Protection ser...,No
1,System,2023-09-22,00:10:01,CLIENT-PC,Microsoft-Windows-Kernel-General,NaN,4,NaN,Info,16,4,2279,The access history in hive \??\C:\Users\Commun...,No
2,Application,2023-09-22,00:09:45,CLIENT-PC,SecurityCenter,NaN,4,Classic,Info,15,4,3266,Updated Windows Defender status successfully t...,No
3,System,2023-09-22,00:09:23,CLIENT-PC,Microsoft-Windows-Kernel-General,NaN,4,NaN,Info,16,4,2278,The access history in hive \??\C:\Users\Commun...,No
4,Application,2023-09-22,00:09:20,CLIENT-PC,Microsoft-Windows-Security-SPP,NaN,2,Classic,NaN,8198,2,3265,License Activation (slui.exe) failed with the ...,No


In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['Message'], df['isMalicious'], test_size=0.2, random_state=42)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Initialize and train a machine learning model (e.g., Random Forest) to predict IsMalicious
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

# Evaluate the model (optional)
print(classification_report(y_test, model.predict(X_test_tfidf)))

              precision    recall  f1-score   support

          No       1.00      1.00      1.00        63
         Yes       1.00      1.00      1.00      4102

    accuracy                           1.00      4165
   macro avg       1.00      1.00      1.00      4165
weighted avg       1.00      1.00      1.00      4165



In [ ]:
#joblib.dump(nb_classifier, 'multinomial_nb_model.pkl')
with open('random_forest_model.pkl', 'wb') as fout:
    pickle.dump((tfidf_vectorizer, model), fout)

In [ ]:
# Now, you can use this trained model to classify logs in a new dataset and identify potentially malicious logs
new_data = pd.read_csv('new_logs.csv')
new_data_tfidf = tfidf_vectorizer.transform(new_data['Message'])
predicted_probabilities = model.predict_proba(new_data_tfidf)

# Determine which logs in the new dataset are potentially malicious (IsMalicious=1)
new_potentially_malicious_logs = []
for i, prob_scores in enumerate(predicted_probabilities):
    is_malicious = prob_scores[1]  # Probability of being malicious (IsMalicious=1)
    if is_malicious >= 0.7:  # You can adjust this threshold as needed
        new_potentially_malicious_logs.append(new_data['Message'].iloc[i])

if new_potentially_malicious_logs:
    print("The new dataset contains potentially malicious logs.")
    for log in new_potentially_malicious_logs:
        print(log)
else:
    print("The new dataset does not contain potentially malicious logs.")
